In [10]:
# data_files = "/home/salman.khan/shehan/workspace_pointing_lmm/MolmoVideo/datasets/pointing_dataset/annotations_davis17_train.jsonl",

data_files="/share/users/shehan/workspace_pointing_lmm/DataPrep/annotations_davis17_train.jsonl"

base_data_dir = '/share/users/shehan/workspace_pointing_lmm/DataPrep'


In [11]:
SYSTEM_PROMPT = "You are given {num_frames} frames from a video. The frame indices are {selected_frame_idxs}. "
PROMPT_TEMPLATES = [
        "Point to {label}\nPlease say 'This isn't in the video.' if it is not in the video.",
        "Point to all occurrences of \"{label}\"",
        "Point to any {label} in the video",
        "Point to any {label} in the video.",
        "Point: Where are the {label}",
        "Show me where the {label} are",
        "Can you show me where the {label} are?",
        "Show me where the {label} are",
        "Show me where a {label} is",
        "Show me where a {label} is.",
        "If there are any {label} in the video? Show me where they are.",
        "Where are the {label}?",
        "Generate a list of points showing where the {label} are.",
        "Find the \"{label}\".",
        "Find a \"{label}\".",
        "Locate all {label}.",
        "Locate an {label}.",
        "Locate a {label}.",
        "Locate every {label}.",
        "Locate {label}.",
        "Locate the {label}.",
        "Object: {label}\nInstruction: Point to the object.",
        "find {label}",
        "find {label}.",
        "Point to every {label}",
        "find any {label} in the picture",
        "Find the {label}",
        "Find any {label}",
        "Point to a {label}",
        "Point to an {label}",
        "Look for {label} in the video and show me where they are.",
        "Help me find an object in the video by pointing to them.\nObject: {label}.",
        "I am looking for {label}, where can they be found in the video?",
        "Can you see any {label} in the video? Point to them.",
        "Point out each {label} in the video.",
        "Point out every {label} in the video.",
        "Point to the {label} in the video.",
        "Locate each {label} in the video.",
        "Can you point out all {label} in this video?",
        "Please find {label} and show me where they are.",
        "If there are any {label} present, indicate their positions.",
        "If there is a {label} present, indicate its positions.",
        "show me all visible {label}",
    ]

In [12]:
from datasets import load_dataset

# Load the dataset from the JSONL file
dataset = load_dataset("json", 
                       data_files=data_files,
                       split="train")

# Print a sample to verify
print(dataset[0])


# %%
dataset


# %%
dataset[0]

# %%
dataset[0]['points']


{'video': 'davis17/00000/frames', 'frame_idxs': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81], 'timestamps': [0.0, 0.03333333333333333, 0.06666666666666667, 0.1, 0.13333333333333333, 0.16666666666666666, 0.2, 0.23333333333333334, 0.26666666666666666, 0.3, 0.3333333333333333, 0.36666666666666664, 0.4, 0.43333333333333335, 0.4666666666666667, 0.5, 0.5333333333333333, 0.5666666666666667, 0.6, 0.6333333333333333, 0.6666666666666666, 0.7, 0.7333333333333333, 0.7666666666666667, 0.8, 0.8333333333333334, 0.8666666666666667, 0.9, 0.9333333333333333, 0.9666666666666667, 1.0, 1.0333333333333334, 1.0666666666666667, 1.1, 1.1333333333333333, 1.1666666666666667, 1.2, 1.2333333333333334, 1.2666666666666666, 1.3, 1.3333333333333333,

{'0': [17.390625, 60.148148148148145],
 '1': [17.2265625, 60.148148148148145],
 '2': [17.14453125, 59.88888888888889],
 '3': [17.0625, 59.629629629629626],
 '4': [16.98046875, 59.629629629629626],
 '5': [16.8984375, 59.629629629629626],
 '6': [16.8984375, 59.37037037037037],
 '7': [16.8984375, 59.37037037037037],
 '8': [16.98046875, 59.37037037037037],
 '9': [17.0625, 59.37037037037037],
 '10': [17.14453125, 59.629629629629626],
 '11': [17.2265625, 59.629629629629626],
 '12': [17.30859375, 59.88888888888889],
 '13': [17.390625, 59.88888888888889],
 '14': [17.47265625, 60.148148148148145],
 '15': [17.5546875, 60.40740740740741],
 '16': [17.63671875, 60.40740740740741],
 '17': [17.80078125, 60.40740740740741],
 '18': [17.8828125, 60.666666666666664],
 '19': [18.046875, 60.666666666666664],
 '20': [18.2109375, 60.92592592592594],
 '21': [18.2109375, 60.40740740740741],
 '22': [18.29296875, 60.666666666666664],
 '23': [18.375, 60.92592592592594],
 '24': [18.375, 61.18518518518518],
 '25': 

In [13]:
def get_points_in_xml_format(points, caption):
    """
    Convert a list of (t, x, y) points to a string in XML format.
    """
    lines = ["<point"]
    # for t, x, y in points:
    for t, x_y in points.items():
        if x_y is None:
            continue
        x, y = x_y
        # Append each point's attributes on a new, indented line
        lines.append(f' t="{int(t)}" x="{x:.1f}" y="{y:.1f}"')
    # Append the alt attribute and the element's text content on the final line
    lines.append(f' alt="{caption}">{caption}</point>')
    formatted_output = "".join(lines)
    
    return formatted_output

# %%
import os
import random
from PIL import Image


num_frames = 4 #TODO: change to 4

def random_augmentation(batch):
    """
    Batch-wise random augmentation.
    For each example in the batch:
      - Randomly selects `num_frames` indices (or all if there are fewer).
      - Loads only the images corresponding to the selected frame indices.
      - Subsets the 'frame_idxs', 'timestamps', and 'points' to only those indices.
    Assumes that each example in the batch has keys:
      'video'      : the directory containing the frames,
      'frame_idxs' : a list of frame indices,
    #   'timestamps' : a list of timestamps,
      'points'     : a list of (t, x, y) values.
    """
    new_batch = {"images": [], "frame_idxs": [], "timestamps": [], "points": [], "question": [], "answer": []}

    # Process each example in the batch
    for i in range(len(batch["video"])):
        video_dir = batch["video"][i]
        frame_idxs = batch["frame_idxs"][i]    # e.g. [0, 1, 2, ...]
        # timestamps = batch["timestamps"][i]
        points = batch["points"][i]
        caption = batch["caption"][i]

        # Determine the indices to select
        if len(frame_idxs) <= num_frames:
            selected_indices = list(range(len(frame_idxs)))
        else:
            # Select a sorted list of unique random indices to maintain temporal order
            selected_indices = sorted(random.sample(range(len(frame_idxs)), num_frames))

        # Subset the metadata based on selected indices
        selected_frame_idxs = [frame_idxs[j] for j in selected_indices]
        # selected_timestamps = [timestamps[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices if j in points]
        # print('points', points)
        selected_points = {int(k): v for k, v in points.items() if int(k) in selected_frame_idxs}
        # print('selected_points', selected_points)
        

        # Load images corresponding to the selected frame indices
        images = []
        for j in selected_indices:
            # Construct file path: assuming images are named as "00000.jpg", "00001.jpg", etc.
            frame_filename = f"{frame_idxs[j]:05d}.jpg"
            frame_path = os.path.join(base_data_dir, video_dir, frame_filename)
            try:
                image = Image.open(frame_path).convert("RGB")
            except Exception as e:
                print(f"Error loading image {frame_path}: {e}")
                image = None
            images.append(image)

        # Add the selected data to the new batch
        new_batch["images"].append(images)
        new_batch["frame_idxs"].append(selected_frame_idxs)
        # new_batch["timestamps"].append(selected_timestamps)
        new_batch["points"].append(selected_points)
        
        question = SYSTEM_PROMPT.format(num_frames=num_frames, selected_frame_idxs=selected_frame_idxs) + random.choice(PROMPT_TEMPLATES).format(label=caption)
        new_batch["question"].append(question)
        
        answer = get_points_in_xml_format(selected_points, caption)
        new_batch["answer"].append(answer)

    return new_batch

# Now, apply the transform to your dataset:
# dataset = your_huggingface_dataset.with_transform(random_augmentation)
augmented_dataset = dataset.with_transform(random_augmentation)



In [14]:

# %%


# %%
augmented_dataset

# %%
augmented_dataset[0:2]


{'images': [[<PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>],
  [<PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>,
   <PIL.Image.Image image mode=RGB size=336x336>]],
 'frame_idxs': [[11, 30, 56, 76], [16, 70, 73, 74]],
 'timestamps': [],
 'points': [{11: [17.2265625, 59.629629629629626],
   30: [18.29296875, 63.0],
   56: [17.96484375, 58.33333333333333],
   76: [16.8984375, 51.333333333333336]},
  {16: [17.63671875, 60.40740740740741],
   70: [17.30859375, 52.11111111111111],
   73: [17.14453125, 51.333333333333336],
   74: [17.0625, 51.333333333333336]}],
 'question': ['You are given 4 frames from a video. The frame indices are [11, 30, 56, 76]. Point: Where are the a brown bear',
  'You are given 4 frames from a video. The frame indices are

In [15]:
dataset = dataset.remove_columns(["timestamps"])


In [16]:
def get_points_in_xml_format(points, caption):
    """
    Convert a list of (t, x, y) points to a string in XML format.
    """
    lines = ["<point"]
    # for t, x, y in points:
    for t, x_y in points.items():
        if x_y is None:
            continue
        x, y = x_y
        # Append each point's attributes on a new, indented line
        lines.append(f' t="{int(t)}" x="{x:.1f}" y="{y:.1f}"')
    # Append the alt attribute and the element's text content on the final line
    lines.append(f' alt="{caption}">{caption}</point>')
    formatted_output = "".join(lines)
    
    return formatted_output

In [25]:
import os
import random
from PIL import Image


num_frames = 4 #TODO: change to 4

def random_augmentation(batch):
    """
    Batch-wise random augmentation.
    For each example in the batch:
      - Randomly selects `num_frames` indices (or all if there are fewer).
      - Loads only the images corresponding to the selected frame indices.
      - Subsets the 'frame_idxs', 'timestamps', and 'points' to only those indices.
    Assumes that each example in the batch has keys:
      'video'      : the directory containing the frames,
      'frame_idxs' : a list of frame indices,
    #   'timestamps' : a list of timestamps,
      'points'     : a list of (t, x, y) values.
    """
    new_batch = {"images": [], "frame_idxs": [], "points": [], "question": [], "answer": []}

    # Process each example in the batch
    for i in range(len(batch["video"])):
        video_dir = batch["video"][i]
        frame_idxs = batch["frame_idxs"][i]    # e.g. [0, 1, 2, ...]
        # timestamps = batch["timestamps"][i]
        points = batch["points"][i]
        caption = batch["caption"][i]

        # Determine the indices to select
        if len(frame_idxs) <= num_frames:
            selected_indices = list(range(len(frame_idxs)))
        else:
            # Select a sorted list of unique random indices to maintain temporal order
            selected_indices = sorted(random.sample(range(len(frame_idxs)), num_frames))

        # Subset the metadata based on selected indices
        selected_frame_idxs = [frame_idxs[j] for j in selected_indices]
        # selected_timestamps = [timestamps[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices]
        # selected_points = [points[j] for j in selected_indices if j in points]
        # print('points', points)
        selected_points = {int(k): v for k, v in points.items() if int(k) in selected_frame_idxs}
        # print('selected_points', selected_points)
        

        # Load images corresponding to the selected frame indices
        images = []
        for j in selected_indices:
            # Construct file path: assuming images are named as "00000.jpg", "00001.jpg", etc.
            frame_filename = f"{frame_idxs[j]:05d}.jpg"
            frame_path = os.path.join(base_data_dir, video_dir, frame_filename)
            try:
                image = Image.open(frame_path).convert("RGB")
            except Exception as e:
                print(f"Error loading image {frame_path}: {e}")
                image = None
            images.append(image)

        # Add the selected data to the new batch
        new_batch["images"].append(images)
        new_batch["frame_idxs"].append(selected_frame_idxs)
        # new_batch["timestamps"].append(selected_timestamps)
        new_batch["points"].append(selected_points)
        
        question = SYSTEM_PROMPT.format(num_frames=num_frames, selected_frame_idxs=selected_frame_idxs) + random.choice(PROMPT_TEMPLATES).format(label=caption)
        new_batch["question"].append(question)
        
        answer = get_points_in_xml_format(selected_points, caption)
        new_batch["answer"].append(answer)
        
    # Ensure all batch elements are properly structured
    assert len(new_batch["images"]) == len(new_batch["frame_idxs"]) == len(new_batch["points"]) == len(new_batch["question"]) == len(new_batch["answer"])

    return new_batch

# Now, apply the transform to your dataset:
# dataset = your_huggingface_dataset.with_transform(random_augmentation)
augmented_dataset = dataset.with_transform(random_augmentation)

In [26]:
augmented_dataset[1]

{'images': [<PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>],
 'frame_idxs': [4, 28, 30, 73],
 'points': {4: [16.98046875, 59.629629629629626],
  28: [18.29296875, 62.22222222222221],
  30: [18.29296875, 63.0],
  73: [17.14453125, 51.333333333333336]},
 'question': 'You are given 4 frames from a video. The frame indices are [4, 28, 30, 73]. find any a brown bear moving in the picture',
 'answer': '<point t="4" x="17.0" y="59.6" t="28" x="18.3" y="62.2" t="30" x="18.3" y="63.0" t="73" x="17.1" y="51.3" alt="a brown bear moving">a brown bear moving</point>'}

In [ ]:
TypeError: Custom formatting function must return a dict of sequences to be able to pick a row, but got 

{'images': [[<PIL.Image.Image image mode=RGB size=336x336 at 0x7FE4EE702F80>, 
             <PIL.Image.Image image mode=RGB size=336x336 at 0x7FE4EE7019C0>, 
             <PIL.Image.Image image mode=RGB size=336x336 at 0x7FE4EE7016C0>, 
             <PIL.Image.Image image mode=RGB size=336x336 at 0x7FE4EE700DC0>]], 
 'frame_idxs': [[9, 35, 71, 72]], 
 'timestamps': [], 
 'points': [{9: [17.0625, 59.37037037037037], 35: [18.375, 63.51851851851852], 71: [17.2265625, 51.85185185185185], 72: [17.14453125, 51.592592592592595]}], 
 'question': ['You are given 4 frames from a video. The frame indices are [9, 35, 71, 72]. Point: Where are the a brown bear'], 'answer': ['<point t="9" x="17.1" y="59.4" t="35" x="18.4" y="63.5" t="71" x="17.2" y="51.9" t="72" x="17.1" y="51.6" alt="a brown bear">a brown bear</point>']}

In [20]:
print(augmented_dataset)

Dataset({
    features: ['video', 'frame_idxs', 'points', 'caption', 'fps'],
    num_rows: 572
})


In [27]:
augmented_dataset[0]

{'images': [<PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>,
  <PIL.Image.Image image mode=RGB size=336x336>],
 'frame_idxs': [43, 54, 58, 70],
 'points': {43: [18.2109375, 64.03703703703704],
  54: [18.12890625, 60.148148148148145],
  58: [17.80078125, 57.03703703703704],
  70: [17.30859375, 52.11111111111111]},
 'question': 'You are given 4 frames from a video. The frame indices are [43, 54, 58, 70]. Where are the a brown bear?',
 'answer': '<point t="43" x="18.2" y="64.0" t="54" x="18.1" y="60.1" t="58" x="17.8" y="57.0" t="70" x="17.3" y="52.1" alt="a brown bear">a brown bear</point>'}